[View in Colaboratory](https://colab.research.google.com/github/leobezerra/automl-pybr14/blob/master/AutoML.ipynb)

# AutoML com Python

Neste notebook, nós vamos criar um classificador para um problema de classificação de digitos de forma automatizada.

Pra isso, nós vamos usar o pacote ```auto-sklearn```, que implementa a abordagem de AutoML chamada AutoSklearn.

O AutoSklearn é uma abordagem baseada em comitês (ensembles), que consegue selecionar e configurar algoritmos de forma automatizada.

Ele recebe como entrada um conjunto de treino rotulado e produz um algoritmo que espera-se que tenha uma boa performance no problema informado.

O AutoSklearn pode ser usado tanto para classificação como para regressão.

Neste exemplo, nós vamos criar um classificador para o problema de classificação de digitos escritos à mão conhecido como MNIST.

### Instalando o auto-sklearn

O pacote ```auto-sklearn``` tem algumas dependências que nós precisamos instalar primeiro:

**Note que o comando abaixo funciona no Colab e em distribuições Linux baseadas em Debian. Se você usa outra plataforma, você precisará instalar essa biblioteca manualmente.**

In [0]:
!apt-get install build-essential swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 12 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (1,297 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 22278 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3.0.12-1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_3.0.12-1_amd64.deb ...
Unpacking s

In [0]:
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   209  100   209    0     0   1514      0 --:--:-- --:--:-- --:--:--  1514
    100% |████████████████████████████████| 163kB 4.5MB/s 
    100% |████████████████████████████████| 2.1MB 9.4MB/s 
    100% |████████████████████████████████| 460kB 5.7MB/s 
  Running setup.py bdist_wheel for xgboost ... - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/94/02/70/61da39c3c6e67d052eb5c618c2b5188adcd6ab0128025f1323
Successfully built xgboost
  Found existing installation: xgboost 0.7.post4
    Uninstalling xgboost-0.7.post4:
      Successfully uninstalled xgboost-0.7.post4
  Running setup.py bdist_wheel for liac-arff ... - done
  Stored in directory: /root/.cache/pip/wheels/81/f0/15/97687f0a23a6859a7ced7e09271d321930c6641c2675d04745
Successfully built liac-arff
    100% |████████████████████████████████| 921k

**Note que o pip espera o nome ```auto-sklearn``` em vez de ```autosklearn```**

In [0]:
!pip install auto-sklearn

    100% |████████████████████████████████| 3.4MB 9.3MB/s 
  Running setup.py bdist_wheel for auto-sklearn ... - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/3f/4e/d9/489ca4cb2f6fd94f58180b0073d15746583f772f25d9178b94
  Running setup.py bdist_wheel for pynisher ... - done
  Stored in directory: /root/.cache/pip/wheels/81/35/cb/37fe9c279ac6e56fc8805e146a431c27550dce1ad868ffa04e
Successfully built auto-sklearn pynisher
  Found existing installation: pynisher 0.5.0
    Uninstalling pynisher-0.5.0:
      Successfully uninstalled pynisher-0.5.0


## Importando as bibliotecas

Para este exemplo, nós vamos usar o módulo de classificação do AutoSklearn.

In [0]:
from autosklearn.classification import AutoSklearnClassifier

Nós também vamos precisar do módulo `metrics` do `scikit-learn` ao avaliar o modelo produzido. Nós vamos usar as ferramentas de acurácia e matriz de confusão.

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix

## Lendo os dados

Nós vamos usar o dataset MNIST disponível no TensorFlow.

**Você também pode baixar este dataset de outras fontes, caso você não tenha (ou não queira ter) o TensorFlow no seu computador.**

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
m=input_data.read_data_sets("MNIST")

[WARNING] [18:04:29:tensorflow] From <ipython-input-7-d32cef2642e0>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
[WARNING] [18:04:29:tensorflow] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
[WARNING] [18:04:29:tensorflow] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please use urllib or similar 

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST/train-images-idx3-ubyte.gz


[WARNING] [18:04:30:tensorflow] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
[WARNING] [18:04:30:tensorflow] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST/t10k-labels-idx1-ubyte.gz


Este dataset vem previamente dividido em subconjuntos de treine e de teste.

**Isto é extremamente importante para garantir que o modelo produzido seja bom em classificar novas amostras e não só as que ele já conhece.** 

### Exemplos de treino

O subconjunto de treino contém duas coleções: ```images``` e  ```labels```.

Vamos verificar o que ```images``` contém:

**A comunidade de aprendizado de máquina convencionou o uso de X para exemplos e y para rótulos.** 

In [0]:
X_train = m.train.images
print("X_train")
print(X_train)

X_train
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Pelo que conseguimos ver, `X_train` é uma matriz com muitos zeros.

Vamos fuçar um pouco mais:

In [0]:
type(X_train)

numpy.ndarray

Se você está acostumado com `pandas`, `scikit-learn` ou outros pacotes de ciência de dados / aprendizado de máquina, você provavelemente já se deparou com um `ndarray` do pacote `numpy`.

Basicamente, é uma forma mais eficiente da `list` do Python. Nós podemos verificar suas dimensões usando o atributo `shape `:

In [0]:
X_train.shape

(55000, 784)

A forma da coleção ```images``` revela que ele contém 55000 imagens, cada uma representada por uma lista de 784 valores.

Cada vetor é o resultado da linearização da imagem original, que contém 28x28 pixels (28 * 28 = 784).

**Esta representação costumava ser bastante comum em problemas de visão computacional antes do surgimento de algoritmos de deep learning. Apesar de ser compatível com vários algoritmos clássicos, esta representação perde a relação espacial vertical entre os pixels.**

Agora vamos ver o que há na coleção ```labels```.

In [0]:
y_train = m.train.labels
print("y_train")
print(y_train)

y_train
[7 3 4 ... 5 6 8]


Desta vez temos um `ndarray` unidimensional, mas o que esses valores significam?

Vamos verificar seu tipo:

In [0]:
type(y_train)

numpy.ndarray

Mais uma vez temos um ```ndarray```,  então podemos ver sua forma:

In [0]:
y_train.shape

(55000,)

A coleção ```labels``` informa a qual classe cada uma das 55000 imagens da coleção ```images``` pertence.

Neste caso, rótulos variam de 0 to 9, uma vez que são esses os dígitos escritos a mão presentes no dataset.

**Nós podemos fazer um pouco de análise descritiva dos nossos dados usando Pandas e bibliotecas de plot. Aqui nós vamos fazer algo simples e verificar apenas a distribuição dos exemplos de treino entre as diferentes classes.**

In [0]:
from pandas import Series
Series(y_train).value_counts()

1    6179
7    5715
3    5638
2    5470
9    5454
0    5444
6    5417
8    5389
4    5307
5    4987
dtype: int64

Note que o dataaset contém muito mais imagens do número 1 do que número 5.

**É possível preprocessar os dados para balancear o número de exemplos de cada classe. Aqui eu não vou fazer isso pra manter o notebook simples, mas em geral isto é útil para a performance dos modelos.**

### Exemplos de teste

Agora vamos checar como é o subconjunto de teste. Novamente, ele contém coleções `images` e `labels`.

In [0]:
X_test = m.test.images
print("X_test")
print(X_test)

X_test
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


De fato, parece exatamente com o conjunto de imagens de treino.

Vamos checar seu tipo:

In [0]:
type(X_test)

numpy.ndarray

É, ```ndarray``` como esperado. E sua forma?

In [0]:
X_test.shape

(10000, 784)

Agora nós temos 10000 exemplos de imagens 28x28 linearizadas.

**Note que a proporção entre os subconjuntos de treino e teste é bastante a favor do treino. Isto poderia ser um problema em uma aplicação real, mas tem sido usado como padrão para este problema em particular.**

Por último, vamos ver o que tem na coleção `labels`.

In [0]:
y_test = m.test.labels
print("y_test")
print(y_test)

y_test
[7 2 1 ... 4 5 6]


Como esperado, temos uma lista de valores que representam a classe à qual cada exemplo de teste pertence.

Vamos confirmar isso:

In [0]:
type(y_test)

numpy.ndarray

In [0]:
y_test.shape

(10000,)

## Criando um classificador com o pacote `autosklearn`

A abordagem AutoML que estamos estudando aqui está tentando resolver um problema conhecido como **CASH** (seleção e configuração de hiperparâmetros combinada de algoritmos, do inglês *combined algorithm selection and hyperparameter configuration*). 

Este problema lida com duas questões ao mesmo tempo:
* qual o melhor algoritmo para o meu problema?
* como ele pode ser configurado para apresentar a melhor performance possível no meu problema?

Lidar com essas duas questões ao mesmo tempo é o que permite que o AutoSklearn identifique um modelo que já é configurado e que se espera que tenha uma boa performance no problema de entrada.

Para isso, nós precisamos fornecer um setup para o AutoSklearn rodar. Nós fazemos isso através dos parâmetros do construtor da classe `AutoSklearnClassifier`. 

Vamos ver alguns dos mais importantes:
* `time_left_for_this_task`: o tempo total que o AutoSklearn terá pra selecionar/configurar algoritmos. Em geral, isso depende de quais algoritmos você permite que o AutoSklearn teste e de quando tempo eles levam pra rodar no seu problema. Na prática, o tempo total deve permitir que o AutoSklearn rode pelo menos 1000 experiments (testar 1000 algoritmos/configurações diferentes).
* `per_run_time_limit`: o tempo máximo que um único algoritmo/configuração pode usar. Novamente, isto depende de quais algoritmos você permite que o AutoSklearn teste e de quando tempo eles levam pra rodar no seu problema. Em geral, você não deve permitir que um único algoritmo/configuração use mais de 1% do seu tempo total de execução, mas dependendo da aplicação esse limite pode ser flexibilizado para 10%.
* `resampling_strategy`: a estratégia usada internamente pelo AutoSklearn para separa entre subconjuntos de treino e validação. Holdout é a mais simples (e a padrão), mas outras técnicas como validação cruzada também podem ser usadas.

O código abaixo configura um `AutoSklearnClassifier` para ter 20min para selecionar/configurar um algoritmo de alta performance, usando holdout como estratégia de validação e limitando cada teste para um máximo de 2 min.

**Note que as configurações padrões do construtor `AutoSklearnClassifier` permitem que o AutoSklearn rode por 1h. Isto ainda não é considerado suficiente em cenários práticos -- você deveria tentar algo entre 24h e e 72h, o que significa deixar o computador trabalhando pra você enquanto você vai passar o fim de semana na praia :D**

In [0]:
automl_20min = AutoSklearnClassifier(time_left_for_this_task=1200, per_run_time_limit=120, resampling_strategy='holdout')
automl = automl_20min

In [0]:
#automl_1h = AutoSklearnClassifier()
#automl = automl_1h

Agora nós dizemos para o objeto `AutoSklearnClassifier` para selecionar o algoritmo configurado que melhor consiga modelar nossos dados:

**Cuidado: o código a seguir vai levar o tempo que você tiver configurado para levar.**

In [0]:
automl.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2018-10-19 18:04:57,111:EnsembleBuilder(1):892b6daec9844b8aceec8dd30f96b266] No models better than random - using Dummy Classifier!
[WARNING] [2018-10-19 18:04:57,150:EnsembleBuilder(1):892b6daec9844b8aceec8dd30f96b266] No models better than random - using Dummy Classifier!
[WARNING] [2018-10-19 18:04:59,161:EnsembleBuilder(1):892b6daec9844b8aceec8dd30f96b266] No models better than random - using Dummy Classifier!
[WARNING] [2018-10-19 18:05:01,168:EnsembleBuilder(1):892b6daec9844b8aceec8dd30f96b266] No models better than random - using Dummy Classifier!
[WARNING] [2018-10-19 18:05:03,187:EnsembleBuilder(1):892b6daec9844b8aceec8dd30f96b266] No models better than random - using Dummy Classifier!
[WARNING] [2018-10-19 18:05:05,208:EnsembleBuilder(1):892b6daec9844b8aceec8dd30f96b266] No models better than random - using Dummy Classifier!
[WARNING] [2018-10-19 18:05:07,228:EnsembleBuilder(1):892b6daec9844b8aceec8dd30f96b266] No models better than random - using Dummy Classifier!

/usr/local/lib/python3.6/dist-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/usr/local/lib/python3.6/dist-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/usr/local/lib/python3.6/dist-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2018-10-19 18:18:09,091:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2018-10-19 18:18:09,091:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger


/usr/local/lib/python3.6/dist-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/usr/local/lib/python3.6/dist-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


AutoSklearnClassifier(delete_output_folder_after_terminate=True,
           delete_tmp_folder_after_terminate=True,
           disable_evaluator_output=False, ensemble_nbest=50,
           ensemble_size=50, exclude_estimators=None,
           exclude_preprocessors=None, get_smac_object_callback=None,
           include_estimators=None, include_preprocessors=None,
           initial_configurations_via_metalearning=25,
           ml_memory_limit=3072, output_folder=None,
           per_run_time_limit=120, resampling_strategy='holdout',
           resampling_strategy_arguments=None, seed=1, shared_mode=False,
           smac_scenario_args=None, time_left_for_this_task=1200,
           tmp_folder=None)

**O objeto referenciado por ```automl``` é um algoritmo configurado e esta classe oferece diferentes métodos. Dê uma olhada na [API do AutoSklearn](http://automl.github.io/auto-sklearn/stable/api.html) :D**

## Avaliando o classificador

Para avaliar quão bom o classificador produzido pelo `AutoSklearnClassifier` é, nós precisamos que usá-lo para predizer rótulos para o subconjunto de teste:

In [0]:
y_predicted = automl.predict(X_test)

Vamos ver qual é a saída do método `predict`:

In [0]:
print("y_predicted")
print(y_predicted)

y_predicted
[7 2 1 ... 4 5 6]


Como esperado, nós recebemos uma lista de rótulos preditos para cada exemplo. Vamos só confirmar isso:

In [0]:
type(y_predicted)

numpy.ndarray

In [0]:
y_predicted.shape

(10000,)

É, nós temos 10000 predições em `y_predicted`.

Agora vamos comparar essas predições com os rótulos reais que nós tínhamos em `y_test`.

Nós podemos fazer isso usando uma **matriz de confusão**.

Se você tem m classes no seu problema, uma matriz de confusão é uma matriz mxm
If you have m classes in your problem, a confusion matrix is an m x m que conta na sua matriz diagonal o número de vezes que a predição estava correta.

Caso contrário, se você tiver um exemplo da classe i predito erroneamente como sendo da classe j, isso é informado na célula (i,j):

In [0]:
confusion_matrix(y_test, y_predicted)

array([[ 970,    0,    1,    0,    0,    3,    2,    1,    3,    0],
       [   0, 1119,    3,    3,    0,    2,    4,    1,    3,    0],
       [   5,    0,  999,    6,    2,    1,    3,   10,    6,    0],
       [   1,    0,   13,  963,    0,   12,    0,    9,    9,    3],
       [   1,    0,    2,    0,  951,    0,    6,    0,    3,   19],
       [   5,    1,    1,   14,    4,  853,    7,    1,    5,    1],
       [   7,    3,    1,    0,    4,    7,  930,    0,    6,    0],
       [   0,    4,   23,    4,    4,    0,    0,  979,    3,   11],
       [   4,    0,    7,   12,    7,    9,    4,    4,  918,    9],
       [   7,    5,    3,   10,   11,    4,    1,    4,    4,  960]])

Claramente, nosso modelo consegue classificar bem a maior parte dos exemplos do subconjunto de testes.

Vamos usar uma medida analítica para quantificar isso:

In [0]:
print("Accuracy score",accuracy_score(y_test, y_predicted))

Accuracy score 0.9642


O resultado acima significa que seus classificador está correto em aproximadamente 96,5% das vezes que você pede para ele classificar um dígito escrito a mão similar aos que existem no dataset MNIST.

## Critical discussion

Dada a pequena quantidade de tempo que nós demos ao AutoSklearn, os resultados ainda são muito bons se você não tem nenhum tipo de background em visão computacional.

Este é exatamente o tipo de cenário pro qual AutoML foi pensado: alguém que tem pouco background em aprendizado de máquina e/ou na aplicação.

No entanto, se você tiver conhecimento especializado, você sabe que é possível obter uma acurácia de 97% usando SVM e de mais de 99% usando deep learning.

Além disso, configurar o `AutoSklearnClassifier` ennvolves a number of decisions that directly influence the quality of the results, such as the validation strategy.

That's why the guys behind the autosklearn package the and AutoML community in general (like me) keep researching this topic -- **feel free to join us :D**